In [26]:
import numpy as np
from numba import jit
from numba import njit
import matplotlib.pyplot as plt

In [27]:
N = 20
L = 10.0
h = 0.002
epsilon = 1
sigma = 1
m = 1
v_0 = 1

T = 4
time = np.arange(0, T, h)
f = open("datos.txt", "w")

In [28]:
print(np.random.rand(1, 2)[0]+1)

[1.50683002 1.11319946]


Definir funciones para el programa:

Notación:
- L, indica la longitud de la caja
- T, tiempo total a calcular
- r, matriz con posiciones de las partículas. Tiene dimensiones (N, 2), con N el número de partículas
- R, matriz con la distancia entre partículas

In [29]:
@jit(nopython = True)
def init_cond():
    r = np.zeros((N, 2))
    v =np.zeros((N, 2))
    for i in range(N):
        r[i] = np.array([i%(L/2)*2, 4*i/L ]) + 2*np.random.rand(1, 2)[0]
        theta = np.random.rand()*2*np.pi
        v[i] = v_0 * np.array([np.sin(theta), np.cos(theta)])
    return r, v

@jit(nopython = True)
def cond_contorno(r):
    if(r[0] > L):
        n = int(r[0]/L)
        r[0] = r[0]-n*L
    if(r[0] < 0):
        n = int(r[0]/L)
        r[0] = r[0] + (n+1)*L
    if(r[1] > L):
        n = int(r[1]/L)
        r[1] = r[1]-L
    if(r[1] < 0):
        n = int(r[1]/L)
        r[1] = r[1] + (n+1)*L
    return r

def lennard_jones(r):
    R = compute_distance(r)
    acc = np.zeros((N, 2))
    for i in range(N):
        for j in range(N):
            if(i!=j):
                norm  = np.linalg.norm(R[i, j])
                if (norm < 3):
                    acc[i] = acc[i] + 4*R[i, j]* epsilon * (6*np.power((sigma/norm), 5) - 12*np.power((sigma/norm), 11))/(norm*m)
    return acc


def compute_distance(r):
    R = np.zeros((N, N, 2))
    for i in range(N-1):
        for j in range(i+1, N):
            R[i, j] = r[j]- r[i]
            norm = np.linalg.norm(R[i, j])
            norm_R = R[i, j]/norm
            angle = np.arccos(np.dot(norm_R, np.array([1, 0])))
            max_length = abs(L/np.cos(angle%(np.pi/4)))
            if(norm > max_length):
                R[i, j] =  -R[i, j] *(max_length - norm) / norm
            R[j, i] = -R[i, j]
    return R


def verlet_algorithm(r, v, a):
    w = np.zeros((N, 2))
    for i in range(N):
        r[i] = r[i] + h*v[i] + h*h*a[i]/2
        print(r[i])
        r[i] = cond_contorno(r[i])
        print(r[i])
        w[i] = v[i] + h*a[i]/2
    print('-')
    a = lennard_jones(r)
    for i in range(N):
        v[i] = w[i] + h*a[i]/2
    return r, v, a

def write_vector(r):
    for i in range(N):
        f.write(f"{r[i, 0]}, {r[i, 1]}\n")
    f.write(f"\n")

In [30]:
r, v = init_cond()
write_vector(r)

# fig=plt.figure(figsize=(10,7)) #Size of the plot
# ax=fig.add_subplot(111)

# plt.scatter(v[:,0], v[:,1])

for i in range(len(time)):
    a = lennard_jones(r)
    r, v, a = verlet_algorithm(r, v, a)
    write_vector(r)

[0.94936052 1.65569115]
[0.94936052 1.65569115]
[2.06282178 1.74072892]
[2.06282178 1.74072892]
[4.24993112 1.12795997]
[4.24993112 1.12795997]
[7.5402618  1.51853483]
[7.5402618  1.51853483]
[9.63119785 2.23846016]
[9.63119785 2.23846016]
[0.29793529 2.03738631]
[0.29793529 2.03738631]
[2.05707868 2.88929885]
[2.05707868 2.88929885]
[4.69272157 3.71272847]
[4.69272157 3.71272847]
[7.69383646 3.44203298]
[7.69383646 3.44203298]
[8.31326274 4.30817894]
[8.31326274 4.30817894]
[0.63373754 5.43208787]
[0.63373754 5.43208787]
[3.13660612 5.42003902]
[3.13660612 5.42003902]
[4.07486329 6.05307059]
[4.07486329 6.05307059]
[7.41316382 6.75691436]
[7.41316382 6.75691436]
[8.66517435 5.78293383]
[8.66517435 5.78293383]
[0.7134593 6.8062572]
[0.7134593 6.8062572]
[3.28093944 7.49136516]
[3.28093944 7.49136516]
[4.42916617 7.80829178]
[4.42916617 7.80829178]
[7.07792794 8.51404309]
[7.07792794 8.51404309]
[9.62032798 9.29561395]
[9.62032798 9.29561395]
-
[0.95235002 1.65253897]
[0.95235002 1.6525